#INTRO
This is the final notebook of 3 with the final focus on the modeling phase and a time series approach to price prediction
If you are at a loss in the process, please visit the previous two notebooks

Attention:
Please take note of the code comments

In [0]:
from pandas_datareader import data as dr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Data Loading Process

In [0]:
start_date = '2000-01-01'
end_date = '2019-01-05'

In [0]:
def getTicks(items):
  symbols = []
  for i in items:
    try:
      data = dr.get_data_yahoo(i, start_date, end_date)
    except Exception as e:
      print("there was a problem with {}".format(i))
    else:
      symbols.append(i)
  return symbols

In [0]:
sample_ticks = [
            'NVS'                
]

adj = getTicks(sample_ticks)

In [0]:
df = dr.get_data_yahoo(adj, start_date, end_date)

In [7]:
df.head()

Attributes,High,Low,Open,Close,Volume,Adj Close
Symbols,NVS,NVS,NVS,NVS,NVS,NVS
Date,,,,,,
2000-01-03,33.154121,32.370071,32.594086,32.594086,133200.0,17.756123
2000-01-04,33.490143,31.978046,31.978046,31.978046,243200.0,17.420527
2000-01-05,33.826164,31.810036,32.118057,32.118057,144600.0,17.496799
2000-01-06,33.378136,31.922043,32.818100,32.818100,579200.0,17.878160
2000-01-07,34.274193,32.706093,33.322132,33.322132,89900.0,18.152739


In [0]:
prices = df.loc[:,('Adj Close',slice(None))]


In [0]:
prices.columns = prices.columns.droplevel(0)

In [10]:
prices.head()

Symbols,NVS
Date,
2000-01-03,17.756123
2000-01-04,17.420527
2000-01-05,17.496799
2000-01-06,17.878160
2000-01-07,18.152739


In [11]:
p = prices.loc['2000-01-01':'2000-04-01']
p.tail()

Symbols,NVS
Date,
2000-03-27,16.108650
2000-03-28,16.078140
2000-03-29,16.017122
2000-03-30,16.108650
2000-03-31,16.688316


In [128]:
pip install tbats

In [129]:
pip install pyramid.arima

  Using cached https://files.pythonhosted.org/packages/5c/84/57422f2a6ade3161c586545e38b518ba1b7ab30ee4a4acc29110c0aba2bc/pyramid_arima-0.9.0-cp36-cp36m-manylinux1_x86_64.whl


In [130]:
pip install prophet

In [0]:
from tbats import TBATS
from pmdarima import auto_arima
from fbprophet import Prophet

MODELS:
Forecasting 30-90 Days

MAPE is the error metric used in all models and called in the rolling_cv(), which splits our time series data for modeling.
MAPE stands for mean average percentage error and it calculates the mean of all errors as percentage change of predicted and true values
MAPE was chosen for it's simplicity to code. it's not uncommon to use this metric for time series forecasting 

In [0]:
## Error metric - MAPE

def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


#Modeling Phase: 
Models were considered appropriate for time series were used and were chosen based on their ranging complexities from the simple average to the lstm.   

Simple Average

In [0]:
def sam(ts):
    avg = ts.mean()
    return(avg)

#rolling_cv() was written to automate the data split and modeling process and does so in a one step manner using a sample column classifier(for training and crossVal) with datetime index conditioning. rolling_cv() predicts one point at a time until it passes through the entire validation set using a small 3 month sample of an 18 year data set.

In [0]:
def rolling_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        prediction = sam(local_train_timeSeries[local_ticker])#****
        df.loc[df.index == crossDate, 'prediction'] = prediction
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]    

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)

In [35]:
rolling_cv(p)

,ticker,MAPE
0,NVS,6.481251


AutoArima: an AutoRegressive model using time lag variables for forecasting and automatically find the optimal parameters p, d, q.
For more details, please visit the capstone report.  

In [0]:
def autoarima(ts,idx):
  
  model = auto_arima(ts, start_p=1, start_q=1, d=0,max_p=5,max_d= 2, max_q=5, error_action='ignore',suppress_warnings=True)
  #trace=True,
  model.fit(ts)

  forecast = model.predict(n_periods=1)# n_periods
  fc = pd.DataFrame(forecast,index = idx,columns=['Prediction'])# valid.index => idx, index must be called with a collection of some kind
  return (fc)

In [0]:
def aa_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        #prediction = autoarima(local_train_timeSeries[local_ticker])
        prediction = autoarima(local_train_timeSeries[local_ticker],df.loc[df.index == crossDate].index)#****
        #print(prediction.values)
        df.loc[df.index == crossDate, 'prediction'] = prediction.values
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]   

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)

In [38]:
p.tail()

Symbols,NVS
Date,
2000-03-27,16.108650
2000-03-28,16.078140
2000-03-29,16.017122
2000-03-30,16.108650
2000-03-31,16.688316


In [39]:
aa_cv(p)

,ticker,MAPE
0,NVS,1.499655


Fourier Transform for AutoArima

In [14]:
test = df.loc[:,(['Open', 'Close','High','Adj Close','Low'],slice(None))]
test.head()

Attributes,High,Low,Open,Close,Adj Close
Symbols,NVS,NVS,NVS,NVS,NVS
Date,,,,,
2000-01-03,33.154121,32.370071,32.594086,32.594086,17.756123
2000-01-04,33.490143,31.978046,31.978046,31.978046,17.420527
2000-01-05,33.826164,31.810036,32.118057,32.118057,17.496799
2000-01-06,33.378136,31.922043,32.818100,32.818100,17.878160
2000-01-07,34.274193,32.706093,33.322132,33.322132,18.152739


In [0]:
test.columns = test.columns.droplevel(1)

Transform data for modeling with fourier transform values

In [0]:
def ftransform(data):
    data = data.reset_index()
    data_FT = data[['Date', 'Adj Close']]
    close_fft = np.fft.fft(np.asarray(data_FT['Adj Close'].tolist()))
    close_fft = np.fft.ifft(close_fft)
    #close_fft
    fft_df = pd.DataFrame({'fft':close_fft})
    #print(fft_df.tail())
    fft_df['absolute'] = fft_df['fft'].apply(lambda x: np.abs(x))
    fft_df['angle'] = fft_df['fft'].apply(lambda x: np.angle(x))
    fft_list = np.asarray(fft_df['fft'].tolist())
    fft_list_m10= np.copy(fft_list) 
    fft_list_m10[100:-100]=0
    # 100 components
    data['Fourier'] = pd.DataFrame(fft_list_m10).apply(lambda x: np.abs(x))
    data = data.set_index('Date')
    data = data[['Fourier']]
    print(data[-10:-1])
    #dataset['absolute'] = dataset['Fourier'].apply(lambda x: np.abs(x))
    return data

In [43]:
dta = ftransform(test)

Attributes    Fourier
Date                 
2018-12-20  73.109093
2018-12-21  72.345284
2018-12-24  71.598831
2018-12-26  73.577797
2018-12-27  72.970222
2018-12-28  74.003105
2018-12-31  74.480492
2019-01-02  72.961540
2019-01-03  73.551765


In [44]:
dta  = dta.loc['2000-01-01':'2000-04-01']
dta.tail()

Attributes,Fourier
Date,
2000-03-27,16.108650
2000-03-28,16.078140
2000-03-29,16.017122
2000-03-30,16.108650
2000-03-31,16.688316


In [45]:
aa_cv(dta)

,ticker,MAPE
0,Fourier,1.499665


TBATS: stands for Trigonometric seasonality, Box-Cox transformation, ARMA errors, Trend and Seasonal components. It is an Autoregressive model fortified with trigonometric and statistical qualities that should account for seasonality. For more details, follow the link: https://medium.com/intive-developers/forecasting-time-series-with-multiple-seasonalities-using-tbats-in-python-398a00ac0e8a

In [0]:
# Fit the model
def tbats(ts):
  estimator = TBATS(seasonal_periods=(7, 30),use_box_cox=False)
  model = estimator.fit(ts)

  model_forecast = model.forecast(steps=1)
  return model_forecast

In [0]:
def tbats_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        #prediction = autoarima(local_train_timeSeries[local_ticker])
        prediction = tbats(local_train_timeSeries[local_ticker])#****
        df.loc[df.index == crossDate, 'prediction'] = prediction
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]    

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)

In [49]:
tbats_cv(p)
# about 10 minutes to complete

,ticker,MAPE
0,NVS,3.126702


FB Prophet: a modeling approach using facebooks import.
It is based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. 
In this case, it is applied to rolling_cv() and performs as a result. It performs better by itself. For more details: https://facebook.github.io/prophet/

In [0]:
def prophet(df,local_ticker):
  fb = df.reset_index()
  fb = fb.rename(columns = {'Date':'ds', local_ticker:'y'})
  m = Prophet()
  m.fit(fb)
  future = m.make_future_dataframe(periods=1)
  forecast = m.predict(future)
  return forecast.iloc[-1][-1]

In [0]:
def fb_cv(data):
  final_results_df = pd.DataFrame()
  
  for local_ticker in data.columns:
    df = data[[local_ticker]]
    df['sample'] = np.where(df.index < '2000-02-15', 'train', 'test')
    df['sample'] = np.where((df.index >= '2000-02-15') & (df.index <= '2000-03-15'), 'crossVal', df['sample'])


    # create empty column to capture one step predictions
    df['prediction'] = np.nan
   
    # Get dates during the cross validation slice and sort them
    crossVal_Dates = df.index[df['sample'] == 'crossVal'].sort_values()
   
    # roll the train window forward by one data point with every iteration and predict next point
    for crossDate in crossVal_Dates:
        local_train_timeSeries = df[[local_ticker]][df.index < crossDate]
        #prediction = autoarima(local_train_timeSeries[local_ticker])
        prediction = prophet(local_train_timeSeries[local_ticker],local_ticker)#****
        df.loc[df.index == crossDate, 'prediction'] = prediction
   
    # filter the predictions from df to calculate error
    crossVal_df = df[df.index.isin(crossVal_Dates)]    

    mape_local_df = (mape(crossVal_df[local_ticker], crossVal_df['prediction']))  
    #return(mean_absolute_percentage_error(crossVal_df[local_ticker], crossVal_df['prediction'])) 
    local_results_df = pd.DataFrame({'ticker' : local_ticker,
                                    'MAPE' : mape_local_df}, index = [0])
    final_results_df = final_results_df.append(local_results_df)
  return (final_results_df)

In [52]:
fb_cv(p)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 23.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 23.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:n_changepoints greater than number of observations.Using 24.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this

,ticker,MAPE
0,NVS,36.877404


#LSTM: departure from AutoRegression in favor of Neural Network Modeling

In [53]:
pip install tensorflow

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Activation

In [0]:
from sklearn.preprocessing import MinMaxScaler

In [16]:
dataset = test.copy()
dataset.head()

Attributes,High,Low,Open,Close,Adj Close
Date,,,,,
2000-01-03,33.154121,32.370071,32.594086,32.594086,17.756123
2000-01-04,33.490143,31.978046,31.978046,31.978046,17.420527
2000-01-05,33.826164,31.810036,32.118057,32.118057,17.496799
2000-01-06,33.378136,31.922043,32.818100,32.818100,17.878160
2000-01-07,34.274193,32.706093,33.322132,33.322132,18.152739


In [0]:
# preferred reverse indexing 
dataset = dataset.reindex(index = dataset.index[::-1])

In [20]:
dataset.index[:10]

DatetimeIndex(['2000-01-03', '2000-01-04', '2000-01-05', '2000-01-06',
               '2000-01-07', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14'],
              dtype='datetime64[ns]', name='Date', freq=None)

Aggregating values over Columns to create one signal/Column vector

In [0]:
# OpenHighLowClose = OHLC
OHLC_avg = p.mean(axis = 1)
# p defined in the first few cells above 

In [22]:
OHLC_avg[:10]

Date
2000-01-03    17.756123
2000-01-04    17.420527
2000-01-05    17.496799
2000-01-06    17.878160
2000-01-07    18.152739
2000-01-10    18.183243
2000-01-11    17.817141
2000-01-12    17.725615
2000-01-13    17.939177
2000-01-14    17.878160
dtype: float64

In [0]:
# prepare ts data
OHLC_avg = np.reshape(OHLC_avg.values, (len(OHLC_avg),1)) 
scaler = MinMaxScaler(feature_range=(0, 1))
OHLC_avg = scaler.fit_transform(OHLC_avg)

# transforming data w/ scaler for lstm model w/o time
# as single element row vectors in a column

In [24]:
OHLC_avg[:10]

array([[0.88034256],
       [0.78632554],
       [0.80769336],
       [0.91453139],
       [0.99145426],
       [1.        ],
       [0.89743671],
       [0.87179575],
       [0.931625  ],
       [0.91453139]])

In [0]:
# train test split
train_OHLC = int(len(OHLC_avg) * 0.75)
test_OHLC = len(OHLC_avg) - train_OHLC
train_OHLC, test_OHLC = OHLC_avg[0:train_OHLC,:], OHLC_avg[train_OHLC:len(OHLC_avg),:]

In [26]:
train_OHLC.shape

(47, 1)

In [27]:
train_OHLC[:10]

array([[0.88034256],
       [0.78632554],
       [0.80769336],
       [0.91453139],
       [0.99145426],
       [1.        ],
       [0.89743671],
       [0.87179575],
       [0.931625  ],
       [0.91453139]])

It turns out that the structure of the time series input data is similar to rolling_cv(), our prediction method will be similar as a result with the except that we lose the last time step value in the next iteration, so that one more relevent/recent time step value is taken into account.
Altogether, we could be take larger time slices but in this case, we have chosen to take a single time step per iteration

In [0]:
# method for transformation into 1d data

# t_step_size == time step size

def new_dataset(dataset, t_step_size):
  data_X, data_Y = [], []
  limit = len(dataset)-t_step_size-1

  for i in range(limit):
    j = i + t_step_size
    #a = dataset.iloc[i:j, 0]
    a = dataset[i:j, 0] #       row vector inputs 
    #print(a)
    data_X.append(a) 
    #b = dataset.iloc[j, 0]
    b = dataset[j, 0]   #       row vectors inputs
    #print(b)
    data_Y.append(b)
  return np.asarray(data_X), np.asarray(data_Y)

In [0]:
# row vectors each with a single element, time slices: t              instead of t   t+1 t+2
trainX, trainY = new_dataset(train_OHLC, 1) #          t+1                       t+3 t+3 t+5
testX, testY = new_dataset(test_OHLC, 1)    #          t+2                       t+6 t+7 t+8
                                            #                                    etc...
# the values are just shifted up so t+1 is the starting position in trainY
# and similarly t+n+1 is not in trainX but in trainY since that is the last
# value to be predicted

In [32]:
trainX[-10:-1]

array([[0.48718032],
       [0.41880426],
       [0.2820516 ],
       [0.27350506],
       [0.29914628],
       [0.30769416],
       [0.2820516 ],
       [0.21367473],
       [0.21367473]])

In [33]:
trainY[10:-1]

array([0.78632554, 0.81196703, 0.72649735, 0.52136702, 0.50427501,
       0.41880426, 0.39316277, 0.25640984, 0.13675186, 0.27350506,
       0.25640984, 0.35042739, 0.26495665, 0.2905984 , 0.31624043,
       0.2820516 , 0.20512819, 0.24786303, 0.1452992 , 0.1111109 ,
       0.23931729, 0.44444522, 0.43589841, 0.45299202, 0.48718032,
       0.41880426, 0.2820516 , 0.27350506, 0.29914628, 0.30769416,
       0.2820516 , 0.21367473, 0.21367473, 0.00854654])

In [158]:
trainX.shape

(45, 1)

In [0]:
# reshape train, test data for input into model
                            
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1])) # (45,1,1) => 
                                                                   # (batch_size,time_steps, inp_dim)
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
t_step_size = 1

# matrix with 1 channel, in 3rd dimension
# input taken in as row vector time slices

In [167]:
trainX[:10]

array([[[0.88034256]],

       [[0.78632554]],

       [[0.80769336]],

       [[0.91453139]],

       [[0.99145426]],

       [[1.        ]],

       [[0.89743671]],

       [[0.87179575]],

       [[0.931625  ]],

       [[0.91453139]]])

The architecture of the LSTM takes time slices over signals/columns/tickers sequentially but since we have only one ticker, which we aggregated from Open Close High Low prices. We have only one element row vectors. t, t+1, t+2,...,t+n for n time slices 

In [0]:
# lstm model


# initializing rnn//
model = Sequential()

# input layer//
# -- input layer with input_shape(1,1) => 
# 1 sequence(single time slice t, not i.e. [t,t+1,t+2]),
# and 1 signal/column/price from a single ticker
# 32 values will be output from a batch of size 1 
model.add(LSTM(32, input_shape=(1, step_size), return_sequences = True))

# hidden layer//
# 32 values get mapped to an output with 16 values 
model.add(LSTM(16))

# output layer//
# finally, 16 values get mapped to a single value in the dense layer 
model.add(Dense(1))

# activation will be 'linear' so all values will be mapped to the values 
# of a line to assist with optimization
model.add(Activation('linear'))

Fitting the model and calculating errors

In [170]:
# compile model and train

model.compile(loss='mean_squared_error', optimizer='adagrad',metrics = ['mae']) 
# compare mae, adam, adagrad
model.fit(trainX, trainY, epochs=50, batch_size=1, verbose=2)

Epoch 1/50
 - 2s - loss: 0.0969 - mean_absolute_error: 0.2339
Epoch 2/50
 - 0s - loss: 0.0360 - mean_absolute_error: 0.1562
Epoch 3/50
 - 0s - loss: 0.0309 - mean_absolute_error: 0.1551
Epoch 4/50
 - 0s - loss: 0.0261 - mean_absolute_error: 0.1389
Epoch 5/50
 - 0s - loss: 0.0220 - mean_absolute_error: 0.1266
Epoch 6/50
 - 0s - loss: 0.0188 - mean_absolute_error: 0.1155
Epoch 7/50
 - 0s - loss: 0.0163 - mean_absolute_error: 0.1072
Epoch 8/50
 - 0s - loss: 0.0142 - mean_absolute_error: 0.0971
Epoch 9/50
 - 0s - loss: 0.0125 - mean_absolute_error: 0.0900
Epoch 10/50
 - 0s - loss: 0.0111 - mean_absolute_error: 0.0824
Epoch 11/50
 - 0s - loss: 0.0101 - mean_absolute_error: 0.0795
Epoch 12/50
 - 0s - loss: 0.0092 - mean_absolute_error: 0.0774
Epoch 13/50
 - 0s - loss: 0.0086 - mean_absolute_error: 0.0721
Epoch 14/50
 - 0s - loss: 0.0081 - mean_absolute_error: 0.0713
Epoch 15/50
 - 0s - loss: 0.0078 - mean_absolute_error: 0.0687
Epoch 16/50
 - 0s - loss: 0.0075 - mean_absolute_error: 0.0684
E

In [171]:
# evaluate model accuracy

mae = model.evaluate(testX, testY, batch_size=16)
print('MAE:', mae)

14/14 [==============================] - 1s 55ms/step
MAE: [0.004918854217976332, 0.05656498670578003]


# Mean Absolute Error and Mean Absolute Percentage Error are differ in formulae. So we will not compare MAPE from the previous models with LSTM. And we'll say an error of .057 or 5.7% is good. 

# To Summarize the previous results with MAPE metrics: Facebook performed the worst using rolling_cv() with 36.88% error and AutoArima performed the best with 1.5% error
